### PREREQUISITES - Set base_dir

In [ ]:
import os
import sys

# Check if running in Google Colab
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    base_dir = "/content/drive/MyDrive/WildFire_RemoteSensing_workshop/WildFire_RemoteSensing/"
    print(f"📂 Running in Colab, base_dir set to: {base_dir}")
else:
    base_dir = ""  # Adjust as needed
    print(f"🖥️ Running locally, base_dir set to: {base_dir}")


# Install Dependencies
Install required Python packages (`tqdm`, `rasterio`) if not already available in the current environment.

In [ ]:
!pip install tqdm rasterio 

# Define Data Directory
Specify the directory containing the NetCDF (`.nc`) files that will be used to build the dataset.


In [ ]:
data_dir = base_dir+"datacubes_2024"

# Import Libraries & Define Functions

- Import all necessary libraries.
- Define the following steps:
  - `get_nc_files()`: finds all `.nc` files in the data directory.
  - `extract_pixels_and_labels()`: extracts pixel-level features and assigns labels (0 = "before fire", 1 = "after fire").
  - `build_dataset()`: loops through files, extracts features, and stacks them into `X` and `y`.
  - `save_dataset_to_csv()`: saves the resulting dataset as a CSV for reuse.
- The process supports Sentinel-2 bands and removes invalid or missing data entries.


In [ ]:
import os
import glob
import xarray as xr
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tqdm import tqdm
import pandas as pd

Set a maximum number of pixels per file

In [ ]:
max_pixels_per_file = 1000

Define aforementioned functions

In [ ]:
def get_nc_files(directory):
    return glob.glob(os.path.join(directory, "*.nc"))

def extract_pixels_and_labels(file_path, label, max_pixels):
    ds = xr.open_dataset(file_path)

    desired_bands = [
        'B01', 'B02', 'B03', 'B04', 'B05', 'B06',
        'B07', 'B08', 'B8A', 'B09', 'B11', 'B12'
    ]

    available_bands = [
        band for band in desired_bands
        if band in ds.data_vars and np.issubdtype(ds[band].dtype, np.number)
    ]

    if not available_bands:
        print(f"⚠️ No valid bands found in {file_path}, skipping.")
        return np.empty((0, len(desired_bands))), np.array([])

    ds = ds[available_bands]

    if "t" in ds.dims:
        ds = ds.isel(t=0)

    da = ds.to_array().transpose("y", "x", "variable")

    pixels = da.values.reshape(-1, da.shape[2]).astype(np.float32, copy=False)
    pixels = pixels[~np.isnan(pixels).any(axis=1)]

    np.random.shuffle(pixels)
    return pixels[:max_pixels], np.full(min(len(pixels), max_pixels), label)

def build_dataset(files, max_pixels):
    X, y = [], []
    for f in tqdm(files, desc="Processing .nc files"):
        label = 0 if "before" in f.lower() else 1
        pixels, labels = extract_pixels_and_labels(f, label, max_pixels)
        X.append(pixels)
        y.append(labels)
    return np.vstack(X), np.hstack(y)

def save_dataset_to_csv(X, y, output_path=base_dir+"DATA/pixel_dataset.csv"):
    df = pd.DataFrame(X, columns=[f"band_{i+1}" for i in range(X.shape[1])])
    df["label"] = y
    df.to_csv(output_path, index=False)
    print(f"✅ Dataset saved to: {output_path}")

Use the functions to build a dataset and save it to a `.csv`

In [ ]:
files = get_nc_files(data_dir)
X, y = build_dataset(files, max_pixels_per_file)
save_dataset_to_csv(X, y,)

# Read from file

Load in your dataset and check that everything looks fine, then use the dataset for analysis.

In [ ]:
import pandas as pd

df=pd.read_csv(base_dir+'DATA/pixel_dataset.csv')
df

In [ ]:
df.describe()

## Band Scatter Plot (B1 vs B12)
Visualize how two spectral bands (band_1 and band_12) vary across burn labels using a 2D scatter plot.

Try changing the band numbers and see the corelation between bands values!

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

plt.scatter(df['band_1'],df['band_12'],alpha=0.5,c=df['label'])

In [ ]:
n_bands = 12
fig, axs = plt.subplots(n_bands, n_bands, figsize=(20, 20))
plt.subplots_adjust(hspace=0.3, wspace=0.3)

Add a custom colormap for visualization.

In [ ]:
cmap = plt.cm.colors.ListedColormap(['#1f77b4', '#ff7f0e'])  # Blue=unburned, Orange=burned

Plot each band combination.

In [ ]:
for i in range(n_bands):
    for j in range(n_bands):
        ax = axs[i, j]
        
        if i <= j:
            ax.remove()
            continue
            
        # Plot scatter points
        scatter = ax.scatter(df['band_'+str(i+1)], df['band_'+str(j+1)], c=df['label'], cmap=cmap, 
                            alpha=0.6, s=10, edgecolors='none')
        
        # Axis labels
        if i == n_bands-1:  # Bottom row
            ax.set_xlabel(  'band_'+str(i+1))
        if j == 0:  # Left column
            ax.set_ylabel('band_'+str(j+1))
            
        ax.set_xticks([])
        ax.set_yticks([])

# Optional: Add colorbar
# cbar = fig.colorbar(scatter, ax=axs, orientation='vertical', fraction=0.02)
# cbar.set_ticks([0.25, 0.75])
# cbar.set_ticklabels(['Unburned', 'Burned'])

plt.suptitle('Band Relationship Scatter Matrix (Burned vs Unburned)', y=0.92)
plt.show()

## NDVI vs NBR Scatter
Calculate NDVI (Normalized Difference Vegetation Index) and NBR (Normalized Burn Ratio) indices, then plot them to evaluate their ability to separate burned and unburned pixels.


In [ ]:
ndvi=(df['band_8']-df['band_4'])/(df['band_8']+df['band_4'])
nbr=(df['band_9']-df['band_12'])/(df['band_9']+df['band_12'])
plt.scatter(ndvi, nbr, c=df['label'], cmap=cmap, alpha=0.6, s=10, edgecolors='none')

# NDVI and NBR Distributions
Plot histograms of NDVI and NBR values separately for burned and unburned classes to visualize statistical separation.


In [ ]:
ndvi = np.clip(ndvi, -1, 1)
nbr = np.clip(nbr, -1, 1)
labels=df['label']

ndvi_burned = ndvi[labels == 1]
ndvi_unburned = ndvi[labels == 0]
nbr_burned = nbr[labels == 1]
nbr_unburned = nbr[labels == 0]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

bins = 50

axs[0].hist(ndvi_unburned, bins=bins, alpha=0.6, color='green', label='Unburned', density=True)
axs[0].hist(ndvi_burned, bins=bins, alpha=0.6, color='red', label='Burned', density=True)
axs[0].set_title('NDVI Distribution by Burn Status')
axs[0].set_xlabel('NDVI')
axs[0].set_ylabel('Density')
axs[0].legend()
axs[0].grid(True)

axs[1].hist(nbr_unburned, bins=bins, alpha=0.6, color='green', label='Unburned', density=True)
axs[1].hist(nbr_burned, bins=bins, alpha=0.6, color='red', label='Burned', density=True)
axs[1].set_title('NBR Distribution by Burn Status')
axs[1].set_xlabel('NBR')
axs[1].set_ylabel('Density')
axs[1].legend()
axs[1].grid(True)

plt.suptitle('Distributions of NDVI and NBR for Burned vs Unburned Pixels', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# Create Dataset For classification

## Reload and Clean Dataset
- Reload pixel data.
- Remove rows with any negative values to clean noise or invalid pixels.


In [ ]:
import pandas as pd

df = pd.read_csv(base_dir+"DATA/pixel_dataset.csv")
df = df[(df >= 0).all(axis=1)]
 
df.describe()

## Prepare Train/Test Sets
- Split data into features (`X`) and labels (`y`).
- Apply stratified train/test split.
- Normalize features using `StandardScaler` to prepare for model training.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop("label", axis=1).values
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("✅ Data loaded and split:")
print(f"  Train shape: {X_train.shape}")
print(f"  Test shape: {X_test.shape}")


## Logistic Regression

Evaluate a baseline linear classifier for binary classification of burned vs. unburned pixels.

Train a logistic regression model on the training set.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

Output evaluation metrics.

In [ ]:
print("Logistic Regression Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Multiple Classification Models
Let's test some other popular classifiers for comparison:
- Random Forest
- Decision Tree
- K-Nearest Neighbors
- Naive Bayes

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

models = {
    "Random Forest": RandomForestClassifier(n_estimators=10, random_state=42),
    "Decision Tree Classifier" : DecisionTreeClassifier( random_state=42),
     "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=3),
     "Naive Bayes": GaussianNB(),
 }


### Train & Evaluate Models
Fit each model on the training data and prints accuracy on the test set.
(*Classification report optionally included but commented out.*)


In [ ]:
from sklearn.metrics import classification_report

for name, model in models.items():
    print(f"\nTraining and evaluating: {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    #print(classification_report(y_test, y_pred))


### Hyperparameter Tuning with GridSearchCV
- Define a parameter grid for `RandomForestClassifier`.
- Run 5-fold cross-validation to find the best combination.
- Evaluate the best model on the test set.


In [ ]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [10, 80],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Best RF params:", grid_search.best_params_)
print("Best RF CV accuracy:", grid_search.best_score_)

best_rf = grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred_rf))


# Stacking Clasifier

Stack different types of popular estimators to combine their performance.

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', LinearSVC(    random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=5)),
]

meta_clf = LogisticRegression(max_iter=1000, random_state=42)

stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_clf,
    cv=5,
    n_jobs=-1,
    passthrough=True
)

stacking_clf.fit(X_train, y_train)

y_pred_stack = stacking_clf.predict(X_test)
print("Stacking Classifier Performance:\n", classification_report(y_test, y_pred_stack))

### Use Stacking Classifier Model to Predict on New Image
Describes application of trained model to classify all pixels in a new `.nc` image file.

In [ ]:
import xarray as xr
import numpy as np
import rasterio
from rasterio.transform import from_origin
import joblib
import os

def classify_nc_to_tiff(nc_file_path, model, scaler, output_tiff_path):
    ds = xr.open_dataset(nc_file_path)
    
    desired_bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06',
                     'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
    ds = ds[[band for band in desired_bands if band in ds.data_vars]]
    
    if "t" in ds.dims:
        ds = ds.isel(t=0)
    
    da = ds.to_array().transpose("y", "x", "variable")
    
    height, width = da.shape[0], da.shape[1]
    coords = ds.coords
    
    pixels = da.values.reshape(-1, da.shape[2])
    nan_mask = np.isnan(pixels).any(axis=1)

    valid_pixels = pixels[~nan_mask]
    valid_pixels_scaled = scaler.transform(valid_pixels)

    predictions = model.predict(valid_pixels_scaled)

    full_pred = np.full((pixels.shape[0],), fill_value=-1, dtype=np.int16)
    full_pred[~nan_mask] = predictions
    classification_map = full_pred.reshape((height, width))

    transform = from_origin(
        float(ds.x[0]), float(ds.y[0]), 
        float(ds.x[1] - ds.x[0]), 
        float(ds.y[0] - ds.y[1])
    )
    
    with rasterio.open(
        output_tiff_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=rasterio.int16,
        crs=ds.rio.crs if hasattr(ds, "rio") else "EPSG:4326",
        transform=transform,
    ) as dst:
        dst.write(classification_map, 1)

    print(f"✅ Saved classification map to: {output_tiff_path}")


In [ ]:
classify_nc_to_tiff(base_dir+"datacubes_2024/fire_224248_after.nc", stacking_clf, scaler, base_dir+"classified_map.tif")

In [ ]:
classify_nc_to_tiff(base_dir+"datacubes_2024/fire_226116_before.nc", stacking_clf, scaler, base_dir+"classified_map_bef.tif")

Use the trained stacking classifier to predict land cover (burned/unburned) from a new `.nc` file and save the result as a `.tif` image.


In [ ]:
import rasterio
import matplotlib.pyplot as plt
with rasterio.open(base_dir+"classified_map_bef.tif") as src:
    image = src.read(1)  # Read the first band
    plt.imshow(image, cmap='viridis')
    plt.title("Predicted Regression Output")
    plt.colorbar(label="Predicted Value")
    plt.axis('off')
    plt.show()

<h2>Lazy Clasifier</h2>

In [ ]:
'''
#%pip install lazypredict

from lazypredict.Supervised import LazyClassifier

# Initialize LazyClassifier
# Consider reducing predictions per model and setting ignore_warnings=True
# for faster execution on potentially smaller datasets
lazy_clf = LazyClassifier(verbose=0, ignore_warnings=False, custom_metric=None)

# Fit the models
models, predictions = lazy_clf.fit(X_train_scaled, X_test_scaled, y_train, y_test)

print("\n--- LazyClassifier Results ---")
models
'''